In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import accuracy_score, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.applications.resnet50 import ResNet50

In [ ]:

# carregar amostras de áudio e rotular com a classe correspondente
path = "/content/drive/MyDrive/Parkinson_LONDON/frases"
samples_with_parkinson = []
samples_without_parkinson = []
for filename in os.listdir(path):
    if "com_parkinson" in filename:
        sample, sr = librosa.load(os.path.join(path, filename))
        samples_with_parkinson.append((sample, 1))
    elif "sem_parkinson" in filename:
        sample, sr = librosa.load(os.path.join(path, filename))
        samples_without_parkinson.append((sample, 0))

# embaralhar as listas
np.random.shuffle(samples_with_parkinson)
np.random.shuffle(samples_without_parkinson)

# criar modelo CNN-2D pré-treinado ResNet-50
base_model = ResNet50(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True)

def extract_features(samples):
  features = []
  for sample in samples:
    x = librosa.feature.melspectrogram(sample[0], sr=22050)
    x = librosa.power_to_db(x, ref=np.max)
    x = np.expand_dims(x, axis=-1)
    x = np.expand_dims(x, axis=0)
    features.append(model.predict(x)[0])
  return features

loo = LeaveOneOut()
X = np.array(samples_with_parkinson + samples_without_parkinson)
y = X[:, 1]
X = X[:, 0]
X_features = extract_features(X)
scores = []
for train_index, test_index in loo.split(X_features):
  # dividir dados em treinamento e teste
  X_train, X_test = X_features[train_index], X_features[test_index]
  y_train, y_test = y[train_index], y[test_index]
  # aumentar dados de treinamento
  datagen.fit(X_train)
    # treinar modelo
  model.fit(datagen.flow(X_train, y_train, batch_size=32),
  epochs=10, steps_per_epoch=len(X_train) // 32)
  # testar modelo
  score = model.evaluate(X_test, y_test)
  scores.append(score[1])

accuracy = np.mean(scores)
print("Accuracy: {:.2f}%".format(accuracy * 100))

IndexError: ignored

In [ ]:
def extract_features(sample):
  x = librosa.feature.melspectrogram(sample, sr=22050)
  x = librosa.power_to_db(x, ref=np.max)
  x = np.expand_dims(x, axis=-1)
  return x


# carregar amostras de áudio e rotular com a classe correspondente
path = "/content/drive/MyDrive/Parkinson_LONDON/frases"
samples_with_parkinson = []
samples_without_parkinson = []
for filename in os.listdir(path):
    if "1" in filename:
        sample, sr = librosa.load(os.path.join(path, filename))
        samples_with_parkinson.append((sample, 1))
    elif "0" in filename:
        sample, sr = librosa.load(os.path.join(path, filename))
        samples_without_parkinson.append((sample, 0))

# embaralhar as listas
np.random.shuffle(samples_with_parkinson)
np.random.shuffle(samples_without_parkinson)

# criar modelo CNN-2D pré-treinado ResNet-50
base_model = ResNet50(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

loo = LeaveOneOut()
X = np.array(samples_with_parkinson + samples_without_parkinson)
y = X[:, 1]
X = X[:, 0]
X = [(sample, label) for sample, label in zip(X, y)]
X_features = extract_features([sample[0] for sample in X])

for train_index, test_index in loo.split(X_features):
    X_train = [X[i][0] for i in train_index]
    y_train = [X[i][1] for i in train_index]
    X_test = [X[i][0] for i in test_index]
    y_test = [X[i][1] for i in test_index]
    X_train_features = extract_features(X_train)
    X_test_features = extract_features(X_test)
    X_train_augmented, y_train_augmented = augment_data(X_train_features, y_train)
    model.fit(X_train_augmented, y_train_augmented, validation_data=(X_test_features, y_test))



# calcular a média das pontuações de precisão
accuracy = np.mean(scores)
print("Accuracy: {:.2f}%".format(accuracy * 100))


<ipython-input-38-49df86983506>:32: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.array(samples_with_parkinson + samples_without_parkinson)


TypeError: ignored

In [ ]:
# Importar os pacotes necessários
import os
import numpy as np
import librosa
import librosa.display
from sklearn.model_selection import LeaveOneOut
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D, Dense
from keras.applications import ResNet50

# Definir o caminho para os arquivos de áudio e os rótulos
data_dir = '/content/drive/MyDrive/Parkinson_LONDON/frases'
audio_files = os.listdir(data_dir)
labels = []
for file in audio_files:
    if '0' in file:
        labels.append(0)
    elif '1' in file:
        labels.append(1)

# Definir as configurações da ResNet-50
input_shape = (224, 224, 3)
num_classes = 2

# Definir a arquitetura da rede
conv_base = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
model = Sequential()
model.add(conv_base)
model.add(GlobalAveragePooling2D())
model.add(Dense(num_classes, activation='softmax'))

# Definir as configurações de treinamento
learning_rate = 0.001
batch_size = 32
epochs = 10

# Definir a técnica de validação cruzada Leave-One-Out
loo = LeaveOneOut()

# Loop para treinar e avaliar o modelo usando Leave-One-Out
accuracy = []
for train_index, test_index in loo.split(audio_files):
    train_files, test_file = [audio_files[i] for i in train_index], audio_files[test_index[0]]
    train_data, train_labels = [], []
    for i, file in enumerate(train_files):
        y, sr = librosa.load(os.path.join(data_dir, file))
        #y = librosa.util.fix_length(y, 120000, mode='constant')
        spectrogram = librosa.feature.melspectrogram(y=y, sr=sr)
        spectrogram = librosa.power_to_db(spectrogram, ref=np.max)
        spectrogram = np.expand_dims(spectrogram, axis=-1)
        spectrogram = np.repeat(spectrogram, 3, axis=-1)
        #spectrogram = librosa.util.pad_center(spectrogram, (input_shape[0], input_shape[1]))
        train_data.append(spectrogram)
        if '0' in file:
            train_labels.append(0)
        elif '1' in file:
            train_labels.append(1)
    test_data, test_labels = [], []
    y, sr = librosa.load(os.path.join(data_dir, test_file))
    #y = librosa.util.fix_length(y, 80000, mode='constant')
    spectrogram = librosa.feature.melspectrogram(y=y, sr=sr)
    spectrogram = librosa.power_to_db(spectrogram, ref=np.max)
    spectrogram = np.expand_dims(spectrogram, axis=-1)
    spectrogram = np.repeat(spectrogram, 3, axis=-1)
   # spectrogram = librosa.util.pad_center(spectrogram, input_shape[:-1])
    test_data.append(spectrogram)
    if '0' in test_file:
      test_labels.append(0)
    elif '1' in test_file:
      test_labels.append(1)
    # Compilar o modelo
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Treinar o modelo
    model.fit(x=np.array(train_data), y=np.array(train_labels), epochs=epochs, batch_size=batch_size)

    # Avaliar o modelo
    test_loss, test_acc = model.evaluate(x=np.array(test_data), y=np.array(test_labels))
    accuracy.append(test_acc)

mean_accuracy = np.mean(accuracy)
print('Mean accuracy:', mean_accuracy)

<ipython-input-43-6a971ab617f1>:75: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  model.fit(x=np.array(train_data), y=np.array(train_labels), epochs=epochs, batch_size=batch_size)


ValueError: ignored

In [1]:
import os
import numpy as np
import librosa
from sklearn.model_selection import LeaveOneOut
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Define o caminho para a pasta com as amostras de áudio
folder_path = '/content/drive/MyDrive/Parkinson_LONDON/frases'
labels = []

# Carrega as amostras de áudio e gera os espectrogramas de frequência
samples = []
for filename in os.listdir(folder_path):
    if filename.endswith('.wav'):
        label = 1 if 'parkinson' in filename else 0
        labels.append(label)
        file_path = os.path.join(folder_path, filename)
        signal, sr = librosa.load(file_path, sr=22050)
        max_length = 6508
        samples = [sample[:max_length] for sample in samples]
        spectrogram = librosa.feature.melspectrogram(y=signal, sr=sr, n_fft=2048, hop_length=512, n_mels=128)
        spectrogram = librosa.power_to_db(spectrogram, ref=np.max)
        spectrogram = spectrogram.astype(np.float32)
        spectrogram = np.expand_dims(spectrogram, axis=-1)
        samples.append(spectrogram)

# Ajusta o tamanho das amostras
samples = pad_sequences(samples, maxlen=max_length, dtype='float32', padding='post', truncating='post', value=0.0)

# Converte as listas para arrays numpy
samples = np.array(samples)
labels = np.array(labels)

# Cria o modelo CNN-1D
model = Sequential()
model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=samples.shape[1:]))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

# Compila o modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Aplica a técnica de cross validation LOO
loo = LeaveOneOut()
accuracies = []
for train_index, test_index in loo.split(samples):
    # Separa os dados de treino e teste
    X_train, X_test = samples[train_index], samples[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    # Treina o modelo
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)
    # Avalia o modelo no dado de teste e armazena a acurácia
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    accuracies.append(accuracy)

# Calcula a acurácia média e o desvio padrão
mean_accuracy = np.mean(accuracies)
std_accuracy = np.std(accuracies)

print("Acurácia média: {:.2f}%".format(mean_accuracy*100))
print("Desvio padrão: {:.2f}%".format(std_accuracy*100))


ValueError: ignored